In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.pooling import AveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop, SGD
from keras import optimizers 
from keras.callbacks import ModelCheckpoint, History, Callback
from keras.layers.advanced_activations import PReLU
from keras.models import load_model
from keras.optimizers import *
from keras import backend as K
K.set_image_data_format('channels_last')

Using CNTK backend


In [2]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validate'
epochs = 50
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(30, 30),
    batch_size=batch_size)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(30, 30),
    batch_size=batch_size)


Found 58852 images belonging to 256 classes.
Found 14718 images belonging to 256 classes.


In [5]:
def create_model(size):
    model = Sequential()
    init = 'he_normal'
    model.add(Conv2D(64, (2, 2), activation = 'relu', name = 'block1_conv1', 
                            kernel_initializer = init, input_shape = (size, size, 3)))
#     model.add(Conv2D(64, (2, 2), activation = 'relu', name = 'block1_conv2', kernel_initializer = init))
    model.add(MaxPooling2D((1, 1), name = 'block1_pool'))

    model.add(Conv2D(128, (2, 2), activation = 'relu', name = 'block2_conv1', kernel_initializer = init))
#     model.add(Conv2D(128, (2, 2), activation = 'relu', name = 'block2_conv2', kernel_initializer = init))
    model.add(MaxPooling2D((1, 1), name = 'block2_pool'))

    model.add(Conv2D(256, (2, 2), activation = 'relu', name = 'block3_conv1', kernel_initializer = init))
    model.add(Conv2D(256, (2, 2), activation = 'relu', name = 'block3_conv2', kernel_initializer = init))
    model.add(MaxPooling2D((2, 2), name = 'block3_pool'))

    model.add(Conv2D(512, (2, 2), activation = 'relu', name = 'block4_conv1', kernel_initializer = init))
    model.add(Conv2D(512, (2, 2), activation = 'relu', name = 'block4_conv2', kernel_initializer = init))
    model.add(MaxPooling2D((2, 2), name = 'block4_pool'))
    
    model.add(Conv2D(512, (2, 2), activation = 'relu', name = 'block5_conv1', kernel_initializer=init))
    model.add(Conv2D(512, (2, 2), activation = 'relu', name = 'block5_conv2', kernel_initializer=init))
    model.add(MaxPooling2D((2, 2), name = 'block5_pool'))
    
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu', kernel_initializer = init, name = 'fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation = 'relu', kernel_initializer = init, name = 'fc2'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation = 'softmax', name = 'predictions'))
    
    optim = Adam(lr = 0.0005)
    
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    return model

model = create_model(size=30)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 29, 29, 64)        832       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 29, 29, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 28, 28, 128)       32896     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 28, 28, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 27, 27, 256)       131328    
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 26, 26, 256)       262400    
_________________________________________________________________
block3_pool (MaxPooling2D)   (None, 13, 13, 256)       0         
__________

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs= epochs,
        validation_data=validation_generator,
        validation_steps=500)

Epoch 1/50
2000/2000 [==============================] - 363s - loss: 5.3796 - acc: 0.0215 - val_loss: 5.3333 - val_acc: 0.0249
Epoch 2/50
2000/2000 [==============================] - 356s - loss: 5.3414 - acc: 0.0231 - val_loss: 5.3178 - val_acc: 0.0228
Epoch 3/50
2000/2000 [==============================] - 355s - loss: 5.2164 - acc: 0.0300 - val_loss: 5.0929 - val_acc: 0.0440
Epoch 4/50
2000/2000 [==============================] - 355s - loss: 4.9278 - acc: 0.0447 - val_loss: 4.6569 - val_acc: 0.0579
Epoch 5/50
2000/2000 [==============================] - 354s - loss: 4.7608 - acc: 0.0507 - val_loss: 4.5058 - val_acc: 0.0701
Epoch 6/50
2000/2000 [==============================] - 354s - loss: 4.6381 - acc: 0.0602 - val_loss: 4.4142 - val_acc: 0.0696
Epoch 7/50
2000/2000 [==============================] - 355s - loss: 4.5439 - acc: 0.0680 - val_loss: 4.3153 - val_acc: 0.0896
Epoch 8/50
2000/2000 [==============================] - 355s - loss: 4.4573 - acc: 0.0760 - val_loss: 4.2214 - 

In [ ]:
model.save_weights('first_try.h5')  # always save your weights after training or during training